Lectura de xml

In [15]:
import pandas as pd
import xml.etree.ElementTree as etree

tree = etree.parse("tg-fal17.xml")
root = tree.getroot()

columns_problem = ["Name", "NrDays", "SlotsPerDay", "NrWeeks"]
columns_optimization = ["Time", "Room", "Distribution", "Student"]
columns_rooms = ["RoomId", "Capacity", "PreviousRoomIdList", "TravelTimeValueList", "UnavailableDaysList", "UnavailableStartList", "UnavailableLengthList", "UnavailableWeeksList"]
columns_courses = ["CourseId", "ConfigIdList"]
columns_configs = ["ConfigId", "SubpartIdList"]
columns_subparts = ["SubpartId", "ClassIdList"]
columns_classes = ["ClassId", "ClassLimit", "ClassParent"]
columns_distributions = ["DistributionId", "DistributionType", "DistributionRequired", "DistributionPenalty", "ClassesList"]
columns_students = ["StudentId", "CoursesList"]
columns_relation_class_room = ["ClassId", "RoomId", "RoomPenalty"]
columns_relation_class_time = ["ClassId", "TimeDays", "TimeStart", "TimeLength", "TimeWeeks", "TimePenalty"]

df_problem = pd.DataFrame(columns = columns_problem)
df_optimization = pd.DataFrame(columns = columns_optimization)
df_rooms = pd.DataFrame(columns = columns_rooms)
df_courses = pd.DataFrame(columns = columns_courses)
df_configs = pd.DataFrame(columns = columns_configs)
df_subparts = pd.DataFrame(columns = columns_subparts)
df_classes = pd.DataFrame(columns = columns_classes)
df_distributions = pd.DataFrame(columns = columns_distributions)
df_students = pd.DataFrame(columns = columns_students)
df_relation_class_room = pd.DataFrame(columns = columns_relation_class_room)
df_relation_class_time = pd.DataFrame(columns = columns_relation_class_time)




df_problem = df_problem.append({"Name": root.attrib.get("name"), "NrDays": root.attrib.get("nrDays"), "SlotsPerDay": root.attrib.get("slotsPerDay"), "NrWeeks": root.attrib.get("nrWeeks")}, ignore_index = True)

for node in root:
    if node.tag == "optimization":
        df_optimization = df_optimization.append({"Time": node.attrib.get("time"), "Room": node.attrib.get("room"), "Distribution": node.attrib.get("distribution"), "Student": node.attrib.get("student")}, ignore_index = True)
    
    elif node.tag == "rooms":
        for room in node:
            roomid = room.attrib.get("id")
            cap = room.attrib.get("capacity")
            prevroomlist = []
            valuelist = []
            dayslist = []
            startlist = []
            lengthlist = []
            weekslist = []
            if len(list(child.tag for child in room.iter() if child is not room)) > 0:
                for child in room:
                    if child.tag == "travel":
                        prevroomid = child.attrib.get("room")
                        value = child.attrib.get("value")
                        prevroomlist.append(prevroomid)
                        valuelist.append(value)
                    elif child.tag == "unavailable":
                        days = child.attrib.get("days")
                        start = child.attrib.get("start")
                        length = child.attrib.get("length")
                        weeks = child.attrib.get("weeks")
                        dayslist.append(days)
                        startlist.append(start)
                        lengthlist.append(length)
                        weekslist.append(weeks)
            if len(prevroomlist) == 0:
                prevroomlist = None
            if len(valuelist) == 0:
                valuelist = None
            if len(dayslist) == 0:
                dayslist = None
            if len(startlist) == 0:
                startlist = None
            if len(lengthlist) == 0:
                lengthlist = None
            if len(weekslist) == 0:
                weekslist = None
            df_rooms = df_rooms.append(pd.Series([roomid, cap, prevroomlist, valuelist, dayslist, startlist, lengthlist, weekslist], index = columns_rooms), ignore_index = True)
            
    elif node.tag == "courses":
        for course in node:
            #print(course.tag)
            courseid = course.attrib.get("id")
            configlist = []
            for config in course:
                #print(config.tag)
                configid = config.attrib.get("id")
                configlist.append(configid)
                subpartlist = []
                for subpart in config:
                    #print(subpart.tag)
                    subpartid = subpart.attrib.get("id")
                    subpartlist.append(subpartid)
                    claselist = []
                    for clase in subpart:
                        #print(clase.tag)
                        claseid = clase.attrib.get("id")
                        claselist.append(claseid)
                        claselimit = clase.attrib.get("limit")
                        claseparent = clase.attrib.get("parent") if clase.attrib.get("parent") is not None else None
                        df_classes = df_classes.append(pd.Series([claseid, claselimit, claseparent], index = columns_classes), ignore_index = True)
                        if len(list(child.tag for child in clase.iter() if child is not clase)) > 0:
                            for child in clase:
                                #print(child.tag)
                                if child.tag == "room":
                                    roomid = child.attrib.get("id")
                                    roompenalty = child.attrib.get("penalty")
                                    df_relation_class_room = df_relation_class_room.append(pd.Series([claseid, roomid, roompenalty], index = columns_relation_class_room), ignore_index = True)
                                elif child.tag == "time":
                                    timedays = child.attrib.get("days")
                                    timestart = child.attrib.get("start")
                                    timelength = child.attrib.get("length")
                                    timeweeks = child.attrib.get("weeks")
                                    timepenalty = child.attrib.get("penalty")
                                    df_relation_class_time = df_relation_class_time.append(pd.Series([claseid, timedays, timestart, timelength, timeweeks, timepenalty], index = columns_relation_class_time), ignore_index = True)
                    df_subparts = df_subparts.append(pd.Series([subpartid, claselist], index = columns_subparts), ignore_index = True)
                df_configs = df_configs.append(pd.Series([configid, subpartlist], index = columns_configs), ignore_index = True)
            df_courses = df_courses.append(pd.Series([courseid, configlist], index = columns_courses), ignore_index = True)
                                    
    elif node.tag == "distributions":
        contador = 1
        for distribution in node:
            #print(distribution.tag)
            distribid = contador
            distribtype = distribution.attrib.get("type")
            distribrequired = distribution.attrib.get("required") if distribution.attrib.get("required") is not None else None
            distribpenalty = distribution.attrib.get("penalty") if distribution.attrib.get("penalty") is not None else None
            listaclases = []
            for clase in distribution:
                #print(clase.tag)
                listaclases.append(clase.attrib.get("id"))
            df_distributions = df_distributions.append(pd.Series([distribid, distribtype, distribrequired, distribpenalty, listaclases], index = columns_distributions), ignore_index = True)
            contador += 1
             
    elif node.tag == "students":
        for student in node:
            studentid = student.attrib.get("id")
            listacourses = []
            for course in student:
                listacourses.append(course.attrib.get("id"))
            df_students = df_students.append(pd.Series([studentid, listacourses], index = columns_students), ignore_index = True)
            
            


print("################################################################################################################")
print("DataFrame Problem")
display(df_problem)
print("################################################################################################################")
print("DataFrame Optimization")
display(df_optimization)
print("################################################################################################################")
print("DataFrame Rooms")
display(df_rooms)
print("################################################################################################################")
print("DataFrame Courses")
display(df_courses)
print("################################################################################################################")
print("DataFrame Configs")
display(df_configs)
print("################################################################################################################")
print("DataFrame Subparts")
display(df_subparts)
print("################################################################################################################")
print("DataFrame Classes")
display(df_classes)
print("################################################################################################################")
print("DataFrame Distributions")
display(df_distributions)
print("################################################################################################################")
print("DataFrame Students")
display(df_students)
print("################################################################################################################")
print("DataFrame RelationClassRoom")
display(df_relation_class_room)
print("################################################################################################################")
print("DataFrame RelationClassTime")
display(df_relation_class_time)
print("################################################################################################################")

################################################################################################################
DataFrame Problem


,Name,NrDays,SlotsPerDay,NrWeeks
0,tg-fal17,7,288,14


################################################################################################################
DataFrame Optimization


,Time,Room,Distribution,Student
0,2,1,20,0


################################################################################################################
DataFrame Rooms


...

################################################################################################################
DataFrame RelationClassRoom


,ClassId,RoomId,RoomPenalty
0,1,4,0
1,4,7,10
2,4,6,10
3,4,1,0
4,4,2,0
...,...,...,...
3067,707,14,0
3068,708,21,0
3069,709,21,0
3070,710,21,0


################################################################################################################
DataFrame RelationClassTime


,ClassId,TimeDays,TimeStart,TimeLength,TimeWeeks,TimePenalty
0,1,1000000,164,27,00001000000000,0
1,2,1000000,190,14,00000111011001,0
2,3,1000000,164,40,00010000100110,0
3,4,1000000,147,30,10000000000000,0
4,4,0100000,147,30,10000000000000,0
...,...,...,...,...,...,...
18379,707,0000100,190,9,00000000000001,0
18380,708,0010000,114,36,11111101011010,0
18381,709,0000100,156,43,00000010100000,0
18382,710,0000100,168,36,00000000000100,0


################################################################################################################


Lectura xml solucion

In [ ]:
import pandas as pd
import xml.etree.ElementTree as etree

tree = etree.parse("solution-tg-fal17.xml")
root = tree.getroot()

columns_sol = ["ClassId", "Days", "Start", "Weeks", "Room"]

df_sol = pd.DataFrame(columns = columns_sol)

for clase in root:
    

Analisis de datos

In [28]:
## Duracion en slots de clases:

df_relation_class_time["TimeLength"] = df_relation_class_time["TimeLength"].astype(int)
print("Promedio:")
display(df_relation_class_time["TimeLength"].mean())
print("Min:")
display(df_relation_class_time["TimeLength"].min())
print("Max:")
display(df_relation_class_time["TimeLength"].max())
print("Count:")
display(df_relation_class_time["TimeLength"].value_counts())
print("Mediana:")
display(df_relation_class_time["TimeLength"].median())
print("Cuartiles:")
display(df_relation_class_time["TimeLength"].quantile([0, 0.25, 0.5, 0.75, 1]))
print("Var:")
display(df_relation_class_time["TimeLength"].var())
print("Std:")
display(df_relation_class_time["TimeLength"].std())

Promedio:


22.904046997389035

Min:


9

Max:


...

0.00     9.0
0.25    20.0
0.50    20.0
0.75    30.0
1.00    96.0
Name: TimeLength, dtype: float64

Var:


86.87628455112772

Std:


9.320744849588348

In [47]:
## Disponibilidad de salas:

for index, row in df_rooms.iterrows():
    print(row["RoomId"], row["UnavailableDaysList"])
    #if row["UnavailableDaysList"] is None:
    #    print(row["RoomId"], row["UnavailableDaysList"])

1 ['0010000', '0000100']
2 ['0010000', '0001000']
3 ['0010000']
4 ['0010000']
5 ['0100000', '0010000', '0001000', '0001000']
6 ['0000100']
7 ['0000100']
8 ['0010000']
9 ['1000000', '0100000', '0010000', '0001000', '0000100']
10 ['1000000', '0100000', '0010000', '0001000', '0000100']
11 None
12 ['1000000', '0100000', '0010000', '0001000', '0000100']
13 None
14 None
15 None
16 None
17 None
18 None
19 None
20 None
21 None
22 None
23 None


In [50]:
## Variabilidad de clases:

#for index, row in df_relation_class_time.iterrows():
#    print(row["ClassId"], row["TimeDays"])
display(df_relation_class_time["ClassId"].value_counts())

334    420
272    420
330    420
78     420
423    350
      ... 
652      1
541      1
157      1
437      1
154      1
Name: ClassId, Length: 711, dtype: int64